In [ ]:
!pip install -q transformers datasets scikit-learn torch

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from datasets import Dataset

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)

os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# --------------------
# Конфигурация
# --------------------
CFG = {
    "model_name": "bert-base-uncased",
    "max_len": 256,
    "batch_train": 16,
    "batch_eval": 32,
    "epochs": 5,
    "lr": 2e-5,
    "seed": 42,
    "use_fp16": torch.cuda.is_available()
}

def compute_metrics(eval_pred):
    """Вычисляет accuracy и f1-score."""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='macro')

    return {
        'accuracy': acc,
        'f1': f1,
    }

In [ ]:
# Загружаем первый датасет
df = pd.read_csv("dataset_news_cut_ready_v2.csv", on_bad_lines='skip')
display(df.head())

,Unnamed: 0.1,Unnamed: 0,title,text,target,text_cnn,text_linear,title_cnn,title_linear
0,0,0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,0,donald trump just couldn t wish all americans ...,donald trump wish americans happy new year lea...,donald trump sends out embarrassing new year’...,donald trump sends embarrassing new year eve m...
1,1,1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,0,house intelligence committee chairman devin nu...,house intelligence committee chairman devin nu...,drunk bragging trump staffer started russian ...,drunk bragging trump staffer started russian c...
2,2,2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",0,"on friday, it was revealed that former milwauk...",friday revealed former milwaukee sheriff david...,sheriff david clarke becomes an internet joke...,sheriff david clarke becomes internet joke thr...
3,3,3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",0,"on christmas day, donald trump announced that ...",christmas day donald trump announced would bac...,trump is so obsessed he even has obama’s name...,trump obsessed even obama name coded website i...
4,4,4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,0,pope francis used his annual christmas day mes...,pope francis used annual christmas day message...,pope francis just called out donald trump dur...,pope francis called donald trump christmas speech


In [ ]:
def run_training_pipeline(df, text_col, label_col, output_dir_name):
    """
    Выполняет полный цикл с разделением на train/val/test
    """
    print(f"--- Запуск конвейера для '{output_dir_name}' ---")

    # 1. Подготовка данных
    df_processed = df[[text_col, label_col]].copy()
    df_processed.columns = ['text', 'label']
    df_processed.dropna(inplace=True)

    train_df, temp_df = train_test_split(
        df_processed,
        test_size=0.3,
        random_state=CFG["seed"],
        stratify=df_processed['label']
    )

    val_df, test_df = train_test_split(
        temp_df,
        test_size=0.5,
        random_state=CFG["seed"],
        stratify=temp_df['label']
    )

    train_ds = Dataset.from_pandas(train_df)
    val_ds = Dataset.from_pandas(val_df)
    test_ds = Dataset.from_pandas(test_df)

    print(f"Размер обучающей выборки: {len(train_ds)}")
    print(f"Размер валидационной выборки: {len(val_ds)}")
    print(f"Размер тестовой выборки: {len(test_ds)}")

    # 2. Токенизация
    tokenizer = AutoTokenizer.from_pretrained(CFG["model_name"])

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding=False, truncation=True, max_length=CFG["max_len"])

    train_tok = train_ds.map(tokenize_function, batched=True)
    val_tok = val_ds.map(tokenize_function, batched=True)
    test_tok = test_ds.map(tokenize_function, batched=True)

    # 3. Обучение
    model = AutoModelForSequenceClassification.from_pretrained(CFG["model_name"], num_labels=2)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    training_args = TrainingArguments(
        output_dir=output_dir_name,
        per_device_train_batch_size=CFG["batch_train"],
        per_device_eval_batch_size=CFG["batch_eval"],
        num_train_epochs=CFG["epochs"],
        learning_rate=CFG["lr"],
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        fp16=CFG["use_fp16"],
        logging_dir=f'./logs/{output_dir_name}',
        logging_steps=100,
        seed=CFG["seed"],
        report_to="none",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_tok,
        eval_dataset=val_tok,
        processing_class=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    print("\nНачало обучения...")
    trainer.train()

    # Загружаем лучшую модель (выбранную по val)
    best_model_path = trainer.state.best_model_checkpoint
    if best_model_path:
        model = AutoModelForSequenceClassification.from_pretrained(best_model_path)

        target_device = training_args.device
        model.to(target_device)

        trainer.model = model

    # Оцениваем на тестовых данных
    test_results = trainer.evaluate(test_tok)

    print(f"--- Конвейер для '{output_dir_name}' завершен ---\n")
    return test_results

In [ ]:
# Запускаем обучение и оценку
results1 = run_training_pipeline(
    df=df,
    text_col='text',
    label_col='target',
    output_dir_name='results_dataset'
)

--- Запуск конвейера для 'results_dataset' ---
Размер обучающей выборки: 31427
Размер валидационной выборки: 6735
Размер тестовой выборки: 6735


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Начало обучения...


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.009600,0.016782,0.995397,0.995389
2,0.010800,0.005969,0.998515,0.998512
3,0.000000,0.006937,0.998515,0.998512
4,0.001700,0.005778,0.998961,0.998959
5,0.000000,0.003199,0.999258,0.999256


--- Конвейер для 'results_dataset' завершен ---



In [ ]:
print("ИТОГОВЫЕ РЕЗУЛЬТАТЫ")
print(f"  -> Accuracy: {results1['eval_accuracy']:.4f}")
print(f"  -> F1-Score: {results1['eval_f1']:.4f}\n")

ИТОГОВЫЕ РЕЗУЛЬТАТЫ
  -> Accuracy: 0.9991
  -> F1-Score: 0.9991

